In [22]:
from app import db
from db import KeyboardPackage
import pandas as pd
import numpy as np
from datetime import timedelta, date
from utils.date import start_of_day, end_of_day
from utils.interval import find_intervals
from analytics.keyboard import get_keyboard_data

In [23]:
device = "PTL_RD_AT_001"
d1 = date(2020, 3, 1)
d2 = date.today() - timedelta(days=1)

In [24]:
data = get_keyboard_data(device)

In [25]:
data

,key_press_count,delete_press_count,enter_press_count,shift_press_count,space_press_count,press_pause_count,pause_length,keystroke_time,press_to_press_time
timestamp,,,,,,,,,
2020-02-19 13:48:00,118,0,2,7,16,0,0.00,0.12,19.83
2020-02-19 13:49:00,80,0,0,3,9,0,0.00,0.12,0.75
2020-02-19 13:50:00,93,0,4,4,11,0,0.00,0.11,0.64
2020-02-19 13:51:00,123,0,3,10,13,0,0.00,0.10,0.48
2020-02-19 13:54:00,17,0,0,2,1,0,0.00,0.14,3.33
...,...,...,...,...,...,...,...,...,...
2020-04-21 07:22:00,60,0,0,4,2,37,1.44,0.14,0.98
2020-04-21 07:23:00,2,0,0,0,0,3,14.84,0.12,20.02
2020-04-22 08:50:00,3,0,1,0,0,3,19.09,0.16,1559.99


In [26]:
data.resample('1d').sum()

,key_press_count,delete_press_count,enter_press_count,shift_press_count,space_press_count,press_pause_count,pause_length,keystroke_time,press_to_press_time
timestamp,,,,,,,,,
2020-02-19,3142,12,77,162,388,74,131.45,160.76,360.51
2020-02-20,6905,46,202,321,843,2622,438.20,6923.37,1429.21
2020-02-21,7636,11,121,410,843,4508,366.96,26.01,345.00
2020-02-22,0,0,0,0,0,0,0.00,0.00,0.00
2020-02-23,0,0,0,0,0,0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
2020-04-19,0,0,0,0,0,0,0.00,0.00,0.00
2020-04-20,0,0,0,0,0,0,0.00,0.00,0.00
2020-04-21,314,1,3,18,8,223,44.96,1.47,191.19


In [3]:
kp = KeyboardPackage

In [13]:
query = db.session.query(kp.frame_start, kp.key_press_count, kp.delete_press_count, kp.enter_press_count, kp.shift_press_count, kp.space_press_count, kp.press_pause_count, kp.pause_length, kp.keystroke_time, kp.press_to_press_time) \
    .filter(kp.device == device)

In [14]:
data = pd.DataFrame(query.all())
data = data.rename(columns=dict(frame_start='timestamp'))

In [18]:
data['date'] = data.timestamp.dt.date

In [20]:
data.groupby('date').sum()

,key_press_count,delete_press_count,enter_press_count,shift_press_count,space_press_count,press_pause_count,pause_length,keystroke_time,press_to_press_time
date,,,,,,,,,
2020-02-19,3142,12,77,162,388,74,131.45,160.76,360.51
2020-02-20,6905,46,202,321,843,2622,438.20,6923.37,1429.21
2020-02-21,7636,11,121,410,843,4508,366.96,26.01,345.00
2020-02-25,4673,35,111,223,524,2113,2928.36,9810.35,1810.79
2020-02-26,188,3,4,13,6,138,105.60,1.37,139.78
2020-02-27,28,0,2,2,1,5,7.84,2721.81,10.49
2020-03-09,36,0,1,1,3,26,2095.91,623.14,2647.86
2020-03-17,271,3,3,21,10,205,56.39,1.13,52.43
2020-03-18,1855,26,112,45,79,1447,136.15,6.52,1353.10


In [21]:
data

,timestamp,key_press_count,delete_press_count,enter_press_count,shift_press_count,space_press_count,press_pause_count,pause_length,keystroke_time,press_to_press_time,date
0,2020-02-19 13:48:00,118,0,2,7,16,0,0.00,0.12,19.83,2020-02-19
1,2020-02-19 13:49:00,80,0,0,3,9,0,0.00,0.12,0.75,2020-02-19
2,2020-02-19 13:50:00,93,0,4,4,11,0,0.00,0.11,0.64,2020-02-19
3,2020-02-19 13:51:00,123,0,3,10,13,0,0.00,0.10,0.48,2020-02-19
4,2020-02-19 13:54:00,17,0,0,2,1,0,0.00,0.14,3.33,2020-02-19
...,...,...,...,...,...,...,...,...,...,...,...
594,2020-04-21 07:22:00,60,0,0,4,2,37,1.44,0.14,0.98,2020-04-21
595,2020-04-21 07:23:00,2,0,0,0,0,3,14.84,0.12,20.02,2020-04-21
596,2020-04-22 08:50:00,3,0,1,0,0,3,19.09,0.16,1559.99,2020-04-22
597,2020-04-23 18:55:00,1,0,0,0,0,2,3059.96,0.07,3059.99,2020-04-23
